In [ ]:
#!pip install selenium
#!apt install chromium-chromedriver
#!cp /usr/lib/chromium-browser/chromedriver /usr/bin
#sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

In [ ]:
from bs4 import BeautifulSoup
import requests
import re
from selenium import webdriver
import sys
import time
import random
import pandas as pd

In [ ]:
def get_course_df():
    course = 'Data+Science'
    df = pd.DataFrame()
    url_temp = 'https://www.coursera.org/search?query={}&page={}&index=prod_all_launched_products_term_optimization'
    base_link = 'https://www.coursera.org'
    headers = {"User-Agent": "Mozilla/5.0 (compatible; Googlebot/2.1; +http://www.google.com/bot.html)"}
    number_of_pages = 20
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

    print('----------------\nSearching for ',course.replace('+',' '))
    
    for start in range(1,number_of_pages+1):

        url=url_temp.format(course,start)
        
        driver.get(url)
        page_source = driver.page_source
        soup=BeautifulSoup(page_source,"html.parser")
        
        print("\r", end="")
        print("Progress: {}%: ".format(start*100//number_of_pages), "▋" * (start*100//number_of_pages // 2), end="")
        sys.stdout.flush()
        
        for i in soup.find_all('a',{'class':"result-title-link",'href':re.compile("/professional-certificates|/specializations|/learn|/projects")}):
            # if tag has attribute of class
            course_link = base_link+i['href']
            r_req=requests.get(course_link,headers=headers)
            time.sleep(random.randrange(3, 7, 1))
            soup_req=BeautifulSoup(r_req.text,"html.parser") 
            try:
                title=i.find('h2',class_="cds-1 card-title css-cru2ji cds-3").text
            except:
                continue
            try:
                organization=i.find('span',{'class':'cds-1 css-1cxz0bb cds-3'}).text
            except:
                continue
            try:
                course_type=i.find('span',{'class':'cds-1 css-yg35ph cds-3'}).text
            except:
                continue
            try:
                rating=i.find('span',{'class':'ratings-text'}).text
            except:
                rating='Unknown'
            try:
                n_student=i.find('span',{'class':'enrollment-number'}).text
            except:
                n_student='Unknown'
            try:
                difficulty=i.find('span',{'class':'cds-1 difficulty css-1vjdgz cds-3'}).text
            except:
                difficulty='Unknown'
            try:
                skill=[]
                for j in soup_req.find_all('span',{'class':'_ontdeqt'}):
                  skill.append(j.text)
            except:
                skill='Unknown'
            
            df = df.append({"search_course":course.replace('+',' '), "course_organization":organization, "course_title":title, 
                            "course_type":course_type, "course_difficulty":difficulty, "course_link":course_link, 
                            "course_students_enrolled":n_student,"skill":skill},
                            ignore_index=True)
            
        

    driver.quit()
            
    print("\n===================\nWeb Scraping Complete!")
    return df

In [ ]:
df = get_course_df()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: use options instead of chrome_options
  if sys.path[0] == '':


----------------
Searching for  Data Science
Progress: 100%:  ▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋
Web Scraping Complete!


In [ ]:
df.head(10)

,search_course,course_organization,course_title,course_type,course_difficulty,course_link,course_students_enrolled,skill
0,Data Science,IBM,IBM Data Science,Professional Certificate,Beginner,https://www.coursera.org/professional-certific...,1.1m,"[Data Science, Deep Learning, Machine Learning..."
1,Data Science,Google,Google Data Analytics,Professional Certificate,Beginner,https://www.coursera.org/professional-certific...,750k,"[Spreadsheet, Data Cleansing, Data Analysis, D..."
2,Data Science,Johns Hopkins University,Data Science,Specialization,Beginner,https://www.coursera.org/specializations/jhu-d...,1.2m,"[Github, Machine Learning, R Programming, Regr..."
3,Data Science,University of Michigan,Applied Data Science with Python,Specialization,Intermediate,https://www.coursera.org/specializations/data-...,800k,"[Text Mining, Python Programming, Pandas, Matp..."
4,Data Science,IBM,Introduction to Data Science,Specialization,Beginner,https://www.coursera.org/specializations/intro...,690k,"[Data Science, Relational Database Management ..."
5,Data Science,IBM,Data Science Fundamentals with Python and SQL,Specialization,Beginner,https://www.coursera.org/specializations/data-...,580k,"[Data Science, Github, Python Programming, Jup..."
6,Data Science,IBM,IBM Data Analyst,Professional Certificate,Beginner,https://www.coursera.org/professional-certific...,720k,"[Microsoft Excel, Python Programming, Data Ana..."
7,Data Science,University of Colorado Boulder,Regression and Classification,Course,Intermediate,https://www.coursera.org/learn/regression-and-...,Unknown,"[Statistics, Data Science, R Programming]"
8,Data Science,IBM,Advanced Data Science with IBM,Specialization,Advanced,https://www.coursera.org/specializations/advan...,110k,"[Data Science, Internet Of Things (IOT), Deep ..."
9,Data Science,CFA Institute,Data Science for Investment Professionals,Specialization,Beginner,https://www.coursera.org/specializations/data-...,1.6k,"[Data Science, Ethics, Machine Learning, Pytho..."


In [ ]:
df.to_csv('Coursera-detail.csv')